# Battle of the Neighborhoods - Cape Town, South Africa

This is the Machine Learning algorithm and supporting code from which the results for my Battelf of the Neighborhoods final project

## Import the required libraries and packages

In [1]:
import pandas as pd #For analysing and manipulating data
import numpy as np #For creating and using vectors
from bs4 import BeautifulSoup #For extracting HTML and XML data
import requests #For handling HTML requests
import xml #For using web data

#To enable usage of JSON files
import json 
from pandas.io.json import json_normalize

#Instal l packages
!pip install tensorflow==2.0.0-alpha0 
!pip install geocoder
!pip install geopy


#To obtain geospatial data
from geopy.geocoders import Nominatim
import geocoder

#For map creation and display
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

#Machine Learning packages
import sklearn as skl
from sklearn.cluster import KMeans


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All packages installed!")

    100% |████████████████████████████████| 79.9MB 436kB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 24.3MB/s ta 0:00:01
    100% |████████████████████████████████| 17.2MB 2.1MB/s eta 0:00:01
    100% |████████████████████████████████| 1.2MB 22.1MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 10.4MB/s ta 0:00:01
    100% |████████████████████████████████| 419kB 31.3MB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 34.6MB/s a 0:00:01
    100% |████████████████████████████████| 61kB 25.6MB/s ta 0:00:01
    100% |████████████████████████████████| 2.2MB 13.9MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 25.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f97024

## Obtain Neighborhood and Postal code information from S A Postal website for Cape Town

In [2]:
scrape = pd.read_html('http://www.sapostalcodes.info/queryPostalCode.php?input_city=CAPE+TOWN', header = 0)

#Create the data frame
cpt = scrape[1]
cpt.head()

,City,Suburb,Street,Box
0,CAPE TOWN,ATHLONE,7760.0,7764.0
1,CAPE TOWN,ATHLONE INDUSTRIA 1,NaN,7764.0
2,CAPE TOWN,ATHLONE INDUSTRIA 2,NaN,7764.0
3,CAPE TOWN,BAKOVEN,NaN,8005.0
4,EASTERN CAPE,BALFOUR,5740.0,5740.0


## Normalize the data and clean it up

In [3]:
#drop all the nieghborhoods without postal codes and remove the decimal from the Stree column
cpt['Street'] = cpt['Street'].astype(str)
cpt.drop(cpt[cpt.Street == 'nan'].index, inplace=True)
cpt.drop(cpt[cpt.City != 'CAPE TOWN'].index, inplace=True)
cpt['Street'] = cpt['Street'].astype(float)
cpt['Street'] = cpt['Street'].astype(int)
cpt.head()

,City,Suburb,Street,Box
0,CAPE TOWN,ATHLONE,7760,7764.0
7,CAPE TOWN,BELHAR,7507,7493.0
8,CAPE TOWN,BELLVILLE,7535,7530.0
9,CAPE TOWN,BERGVLIET,7864,7945.0
10,CAPE TOWN,BO-KAAP,8001,8001.0


In [4]:
#Drop the Box column
cpt.drop("Box", axis=1, inplace=True)
cpt.head()

,City,Suburb,Street
0,CAPE TOWN,ATHLONE,7760
7,CAPE TOWN,BELHAR,7507
8,CAPE TOWN,BELLVILLE,7535
9,CAPE TOWN,BERGVLIET,7864
10,CAPE TOWN,BO-KAAP,8001


In [5]:
#Rename the Suburb and Street columns
cpt.rename(columns={"Suburb": "Neighborhood", "Street": "PostalCode"}, inplace=True)
cpt.head()

,City,Neighborhood,PostalCode
0,CAPE TOWN,ATHLONE,7760
7,CAPE TOWN,BELHAR,7507
8,CAPE TOWN,BELLVILLE,7535
9,CAPE TOWN,BERGVLIET,7864
10,CAPE TOWN,BO-KAAP,8001


In [6]:
#Reset the tuple numbers to run in order
cpt = cpt.reset_index(drop=True)
cpt.head()

,City,Neighborhood,PostalCode
0,CAPE TOWN,ATHLONE,7760
1,CAPE TOWN,BELHAR,7507
2,CAPE TOWN,BELLVILLE,7535
3,CAPE TOWN,BERGVLIET,7864
4,CAPE TOWN,BO-KAAP,8001


## Obtain the Geospatial for the Neighborhoods from a CSV file and merge on PostalCode column

In [7]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Latitude,Longitude
0,7760,-33.965260,18.501795
1,7507,-33.947064,18.610140
2,7535,-33.881273,18.626469
3,7864,-34.047938,18.454311
4,8001,-33.921118,18.421670


In [8]:
#Merge the two files
cpt_geo_pc=pd.merge(cpt, cpt_geo, how='left', on='PostalCode')
cpt_geo_pc.head()

,City,Neighborhood,PostalCode,Latitude,Longitude
0,CAPE TOWN,ATHLONE,7760,-33.965260,18.501795
1,CAPE TOWN,BELHAR,7507,-33.947064,18.610140
2,CAPE TOWN,BELLVILLE,7535,-33.881273,18.626469
3,CAPE TOWN,BERGVLIET,7864,-34.047938,18.454311
4,CAPE TOWN,BO-KAAP,8001,-33.921118,18.421670


In [9]:
#Check the amount of neighborhoods


print('Cape Town has {} Neighborhoods.'.format(len(cpt_geo_pc['PostalCode'].unique()),cpt_geo_pc.shape[0]))

Cape Town has 62 Neighborhoods.


## Create a Map of the Cape town area

In [10]:
#obtain the Coordinates for Cape Town
address = 'Cape Town, South Africa'

geolocator = Nominatim(user_agent="capetown_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The Cape Town coordinates are {}, {}.'.format(latitude, longitude))

The Cape Town coordinates are -33.928992, 18.417396.


In [120]:
#Create map
map_cpt = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, Neighborhood, PostalCode in zip(cpt_geo_pc['Latitude'], cpt_geo_pc['Longitude'], cpt_geo_pc['Neighborhood'], cpt_geo_pc['PostalCode']):
    label = '{}, {}'.format(Neighborhood, PostalCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.7).add_to(map_cpt) 

map_cpt

## Add Foursquare credientials and version (hidden cell)

In [12]:
# The code was removed by Watson Studio for sharing.

## Set the category for Sushi restaurants, make the Foursquared call and obtain the JSON file

In [13]:
#Set the category code

query = "sushi"

In [26]:
def getNearbysushiVenues(names, latitudes, longitudes, radius=100000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            v, 
            lat, 
            lng,
            'Sushi',
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results if v['categories']])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Create the Dataframe showing the different venues from the JSON file

In [28]:
sushi_venues = getNearbysushiVenues(cpt_geo_pc['Neighborhood'],cpt_geo_pc["Latitude"],cpt_geo_pc['Longitude'],2000)
sushi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ATHLONE,-33.965260,18.501795,Hongs Sushi,-33.966223,18.492854,Asian Restaurant
1,ATHLONE,-33.965260,18.501795,Edo Sushi,-33.977238,18.483823,Sushi Restaurant
2,BELLVILLE,-33.881273,18.626469,Sushi Suki,-33.876608,18.628815,Asian Restaurant
3,BELLVILLE,-33.881273,18.626469,Yuz Sushi,-33.880080,18.635716,Sushi Restaurant
4,BELLVILLE,-33.881273,18.626469,Jasons Sushi,-33.873840,18.619392,Sushi Restaurant


## Perform One Hot encoding

In [40]:
#Group the data by Neighborhood, perform a count and drop rows that have no Neighborhood name and duplicates
sushi_venues.drop(sushi_venues[sushi_venues.Neighborhood == 'nan'].index, inplace=True)
sushi_venues.drop(sushi_venues[sushi_venues.Neighborhood == 'HIGH CAPE'].index, inplace=True)
sushi_venues.drop(sushi_venues[sushi_venues.Neighborhood == 'HOUTBAAI'].index, inplace=True)
sushi_venues.drop(sushi_venues[sushi_venues.Neighborhood == 'NUWELAND'].index, inplace=True)
sushi_venues.drop(sushi_venues[sushi_venues.Neighborhood == 'SEEPUNT'].index, inplace=True)
sushi_venues.drop(sushi_venues[sushi_venues.Neighborhood == 'VISHOEK'].index, inplace=True)
sushi_venues.drop(sushi_venues[sushi_venues.Neighborhood == 'KAMPSBAAI'].index, inplace=True)
sushi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ATHLONE,2,2,2,2,2,2
BELLVILLE,5,5,5,5,5,5
BERGVLIET,4,4,4,4,4,4
BO-KAAP,90,90,90,90,90,90
BOTHASIG,2,2,2,2,2,2
CAMPS BAY,2,2,2,2,2,2
CLAREMONT,9,9,9,9,9,9
CONSTANTIA,1,1,1,1,1,1
DURBANVILLE,4,4,4,4,4,4


In [56]:
#Display the total unique Sushi restaurants in Cape Town
print('There are {} unique sushi venues in {} categories.'.format(
    len(sushi_venues['Venue'].unique()),len(sushi_venues['Venue Category'].unique())))

There are 161 unique sushi venues in 12 categories.


In [59]:
# one hot encoding
sushi_onehot = pd.get_dummies(sushi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sushi_onehot['Neighborhood'] = sushi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sushi_onehot.columns[-1]] + list(sushi_onehot.columns[:-1])
sushi_onehot = sushi_onehot[fixed_columns]

sushi_onehot.head()

,Neighborhood,Asian Restaurant,Chinese Restaurant,Dim Sum Restaurant,Food,Food & Drink Shop,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Nightclub,Seafood Restaurant,Sushi Restaurant,Thai Restaurant
0,ATHLONE,1,0,0,0,0,0,0,0,0,0,0,0
1,ATHLONE,0,0,0,0,0,0,0,0,0,0,1,0
2,BELLVILLE,1,0,0,0,0,0,0,0,0,0,0,0
3,BELLVILLE,0,0,0,0,0,0,0,0,0,0,1,0
4,BELLVILLE,0,0,0,0,0,0,0,0,0,0,1,0


## Group the encoded data by Neighborhood

In [60]:
sushi_grouped = sushi_onehot.groupby('Neighborhood').mean().reset_index()
sushi_grouped

,Neighborhood,Asian Restaurant,Chinese Restaurant,Dim Sum Restaurant,Food,Food & Drink Shop,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Nightclub,Seafood Restaurant,Sushi Restaurant,Thai Restaurant
0,ATHLONE,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.500000,0.00
1,BELLVILLE,0.200000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.600000,0.00
2,BERGVLIET,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.750000,0.25
3,BO-KAAP,0.066667,0.000000,0.000000,0.0,0.066667,0.033333,0.000000,0.000000,0.00,0.066667,0.766667,0.00
4,BOTHASIG,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.500000,0.00
5,CAMPS BAY,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,1.000000,0.00
6,CLAREMONT,0.111111,0.000000,0.111111,0.0,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.666667,0.00
7,CONSTANTIA,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,1.000000,0.00
8,DURBANVILLE,0.250000,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.00
9,EDGEMEAD,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,1.000000,0.00


## Find the top 10 sushi venues for each neighborhood

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
tsv = pd.DataFrame(columns=columns)
tsv['Neighborhood'] = sushi_grouped['Neighborhood']

for ind in np.arange(sushi_grouped.shape[0]):
    tsv.iloc[ind, 1:] = return_most_common_venues(sushi_grouped.iloc[ind, :], num_top_venues)

tsv['Cluster Labels'] = kmeans.labels_    
    
tsv.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,ATHLONE,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,1
1,BELLVILLE,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1
2,BERGVLIET,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,0
3,BO-KAAP,Sushi Restaurant,Seafood Restaurant,Food & Drink Shop,Asian Restaurant,Italian Restaurant,Thai Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Food,0
4,BOTHASIG,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,1


## Perform K-Means Clustering

In [111]:
# set clusters
kclusters = 5

kmc = sushi_grouped.drop('Neighborhood', 1)

# run clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kmc)

# check cluster labels
kmeans.labels_[0:10]

array([1, 1, 0, 0, 1, 2, 0, 2, 1, 2], dtype=int32)

In [115]:
#Merge the Dataframes
kmc_merged = sushi_venues

# merge capetown_grouped with cpt_df to add latitude/longitude for each neighborhood
kmc_merged = kmc_merged.join(tsv.set_index('Neighborhood'), on='Neighborhood')
kmc_merged = kmc_merged.reset_index(drop=True)
kmc_merged # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,ATHLONE,-33.965260,18.501795,-30.936834,-33.966223,18.492854,Asian Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,1.0
1,ATHLONE,-33.965260,18.501795,-30.956880,-33.977238,18.483823,Sushi Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,1.0
2,BELLVILLE,-33.881273,18.626469,-30.502597,-33.876608,18.628815,Asian Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
3,BELLVILLE,-33.881273,18.626469,-30.499169,-33.880080,18.635716,Sushi Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
4,BELLVILLE,-33.881273,18.626469,-30.509252,-33.873840,18.619392,Sushi Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
5,BELLVILLE,-33.881273,18.626469,-30.483611,-33.862006,18.633199,Sushi Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
6,BELLVILLE,-33.881273,18.626469,-30.483839,-33.862309,18.633274,Chinese Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
7,BERGVLIET,-34.047938,18.454311,-31.177133,-34.040020,18.456514,Thai Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,0.0
8,BERGVLIET,-34.047938,18.454311,-31.204090,-34.063270,18.452807,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,0.0
9,BERGVLIET,-34.047938,18.454311,-31.191963,-34.042130,18.443795,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,0.0


## Create a Map displaying the above

In [119]:
#Create map
map_cluster = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to map
for lat, lng, Neighborhood in zip(kmc_merged['Neighborhood Latitude'], kmc_merged['Neighborhood Longitude'], kmc_merged['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, PostalCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.7).add_to(map_cpt) 

map_cpt

## Examine the Clusters

### Cluster 1

In [122]:
kmc_merged.loc[kmc_merged['Cluster Labels'] == 0, kmc_merged.columns[[1] + list(range(5, kmc_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
7,-34.047938,18.456514,Thai Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,0.0
8,-34.047938,18.452807,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,0.0
9,-34.047938,18.443795,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,0.0
10,-34.047938,18.449196,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,0.0
11,-33.921118,18.417487,Sushi Restaurant,Sushi Restaurant,Seafood Restaurant,Food & Drink Shop,Asian Restaurant,Italian Restaurant,Thai Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Food,0.0
12,-33.921118,18.417999,Food & Drink Shop,Sushi Restaurant,Seafood Restaurant,Food & Drink Shop,Asian Restaurant,Italian Restaurant,Thai Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Food,0.0
13,-33.921118,18.421991,Sushi Restaurant,Sushi Restaurant,Seafood Restaurant,Food & Drink Shop,Asian Restaurant,Italian Restaurant,Thai Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Food,0.0
14,-33.921118,18.410465,Sushi Restaurant,Sushi Restaurant,Seafood Restaurant,Food & Drink Shop,Asian Restaurant,Italian Restaurant,Thai Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Food,0.0
15,-33.921118,18.420636,Sushi Restaurant,Sushi Restaurant,Seafood Restaurant,Food & Drink Shop,Asian Restaurant,Italian Restaurant,Thai Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Food,0.0
16,-33.921118,18.420073,Sushi Restaurant,Sushi Restaurant,Seafood Restaurant,Food & Drink Shop,Asian Restaurant,Italian Restaurant,Thai Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Food,0.0


### Cluster 2

In [123]:
kmc_merged.loc[kmc_merged['Cluster Labels'] == 1, kmc_merged.columns[[1] + list(range(5, kmc_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,-33.965260,18.492854,Asian Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,1.0
1,-33.965260,18.483823,Sushi Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,1.0
2,-33.881273,18.628815,Asian Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
3,-33.881273,18.635716,Sushi Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
4,-33.881273,18.619392,Sushi Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
5,-33.881273,18.633199,Sushi Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
6,-33.881273,18.633274,Chinese Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0
101,-33.859660,18.544829,Asian Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,1.0
102,-33.859660,18.546940,Sushi Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,1.0
138,-33.830025,18.647940,Asian Restaurant,Chinese Restaurant,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,1.0


### Cluster 3

In [124]:
kmc_merged.loc[kmc_merged['Cluster Labels'] == 2, kmc_merged.columns[[1] + list(range(5, kmc_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
103,-33.951298,18.377194,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
104,-33.951298,18.377194,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
137,-34.026883,18.425341,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
142,-33.872667,18.546940,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
161,-33.868335,18.509757,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
162,-33.868335,18.510070,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
163,-33.881595,18.546940,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
189,-33.908171,18.580503,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
190,-34.023665,18.470462,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0
191,-34.061312,18.452807,Sushi Restaurant,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,2.0


### Cluster 4

In [125]:
kmc_merged.loc[kmc_merged['Cluster Labels'] == 3, kmc_merged.columns[[1] + list(range(5, kmc_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
143,-34.134088,18.430755,Food,Food,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Dim Sum Restaurant,3.0
144,-34.134088,18.430755,Food,Food,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Nightclub,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Dim Sum Restaurant,3.0


### Cluster 5

In [126]:
kmc_merged.loc[kmc_merged['Cluster Labels'] == 4, kmc_merged.columns[[1] + list(range(5, kmc_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
188,-34.017221,18.506424,Nightclub,Nightclub,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Food & Drink Shop,Food,Dim Sum Restaurant,4.0
